In [9]:
!pip install -I -q folium==0.8.3
import pandas as pd
import folium
import geojson
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# sanity check- should be 0.8.3
folium.__version__

'0.8.3'

In [ ]:
# read in csv data
socio = pd.read_csv("IRIS_SOCIOLOGIE.csv", index_col=0)
socio.IRIS = socio.IRIS.astype(str) # convert IRIS var to str for correct joining with geojson
socio.head()

In [ ]:
# function to display maps
def embed_map(m):
    from IPython.display import IFrame
    m.save('index.html')
    return IFrame('index.html', width='100%', height='1000px')

In [ ]:
geo_path = "iris-sociologie.geojson"

# load geojson
g = geojson.load(open("iris-sociologie.geojson"))

# load geojson in GeoDataFrame (for EDA)
geo = gpd.read_file(geo_path)
geo.head()

In [ ]:
# create simple map with geojson outlines of arondissmonts
m = folium.Map(location=[48.6, 2.4], zoom_start=10)

folium.GeoJson(g).add_to(m)

embed_map(m)

# click on the whitespace to the left of the map to toggle scrolling


In [ ]:
# list of column names for choropleth reference
socio.columns

In [ ]:
# create a choropleth overlay for C12_MEN variable
m = folium.Map(location=[48.8, 2.3], zoom_start=11)

folium.Choropleth(
 geo_data=g, # geojson 
 data=socio, # df
 columns=['IRIS', 'C12_MEN'],   # first col must be IRIS, second col sets colors for overlay
 key_on='feature.properties.iris',   # key to match csv data with geojson areas
 fill_color='BuGn').add_to(m)

folium.LayerControl().add_to(m)

embed_map(m)

# click on the whitespace to the left of the cell to toggle scrolling
